In [3]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
from main.packages.mine_generic import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import xgboost as xgb
import optuna
from sklearn.pipeline import make_pipeline

In [4]:
cat = 'food'
hicp_all_path = 'data/preprocessed/hicp_yoy.csv'
hicp_class_path = 'data/HICP_COICOP10s.xlsx'
hicp_cat_path = f'data/preprocessed/{cat}_yoy_infl.csv'
category = 'Food'

HICP_monthly, HICP_class, HICP_cat = import_data_all(hicp_all_path=hicp_all_path,
                                                     hicp_class_path=hicp_class_path,
                                                     hicp_cat_path=hicp_cat_path)

cat_df = split_into_category(category=category,
                             HICP_class=HICP_class,
                             HICP_monthly=HICP_monthly)

Number of items in Food group:  180
Horizon: 2
Training predictor period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Training dependent variable period: 1997-03-31 00:00:00 to 2015-12-31 00:00:00


/Users/lananhnguyen/Desktop/thesis/thesis_code/main/packages/load.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df.fillna(0, inplace=True)


In [ ]:
h = 2
X_cat_train, y_cat_train = split_train_set(X_df=cat_df,
                                           y=HICP_cat,
                                           h = h)

# Hyperparameter tuning:

belong to each specific model

In [19]:
import datetime
#from optuna_dashboard import run_server

#storage = optuna.storages.InMemoryStorage()

# 1. Define an objective function to be maximized.

def objective(trial, X_cat_train, y_cat_train, k_fold = 5):
    """
    For XBG hyperparam tuning
    """
    params = {
        "verbosity": 0,
        'objective': 'reg:squarederror',
        'booster': 'gbtree',
        'eval_metric': 'mae',
        'tree_method': 'hist',
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'num_boosted_rounds': 100000,
        'early_stopping_rounds': 100,

        'gamma': trial.suggest_float('gamma', 0.0, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 10.0),

        'lambda': trial.suggest_float('lambda', 0.01, 1),
        'alpha': trial.suggest_float('alpha', 0.01, 0.5),

        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
    }
    tscv = TimeSeriesSplit(n_splits=k_fold, test_size=24)

    ## Do it explicitly via xgb API:
    cv_mae = [None]*k_fold

    # Add pruning:
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-mae")
        
    for i, (train_index, test_index) in enumerate(tscv.split(X_cat_train, y_cat_train)):
        # split data:
        X_train, X_test = X_cat_train.iloc[train_index], X_cat_train.iloc[test_index]
        y_train, y_test = y_cat_train.iloc[train_index], y_cat_train.iloc[test_index]
        
        # standard scaler:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Transform data into xgboost type:
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_test, label=y_test)

        # Train model:
        model = xgb.train(params, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback], verbose_eval=False)
        
        y_test_pred = model.predict(dvalid)
        cv_mae[i] = mean_absolute_error(y_test, y_test_pred)
        # saving the individual fold holdout metrics 
        # uncomment this line if you don't want this
        #trial.set_user_attr('split_mae', cv_mae)

    # return the mean of all 5 folds        
    return np.mean(cv_mae)

    ## Do it implicitly via scikit-learn
    # Create a pipeline for preprocessing before XGB:
        
    #model = XGBRegressor(**params) #, callbacks=[pruning_callback]
    #pipeline_ = make_pipeline(StandardScaler(), model)

    # Use cross_val_score to get the mean MAE over multiple splits:
    # here it'll split data as tscv, 
    # then for each fold: it'll scale training data, fit model, then scale validation set, get prediction and put out MAE
    #scores = -cross_val_score(pipeline_, X_cat_train, y_cat_train, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=0)
    #print(scores)
    #return scores.mean()

# 2. Create a study object and optimize the objective function.
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)

sampler = optuna.samplers.TPESampler(seed=1234)

study = optuna.create_study(
    pruner=pruner, 
    direction='minimize',
    sampler=sampler
    #storage=storage
)
study.optimize(objective, n_trials=1000)
#run_server(storage, host="localhost", port=8080)
print("Number of finished trials: {}".format(len(study.trials)))


# Get results:
results = study.trials_dataframe(attrs=('number','value','duration', 'params'))
results = results.rename(columns={'value':'mae'})
results['duration'] = results['duration']/np.timedelta64(1, 's')
results = results.sort_values(by='mae',ascending=True).iloc[:100, :]
results.to_csv(f'model/mae_xgboost_h_{h}.csv',index=False)

best_params = study.best_params
best_trial = study.best_trial


[I 2024-04-05 18:42:59,741] A new study created in memory with name: no-name-43672afc-542e-4081-b013-9327cbec009a
[I 2024-04-05 18:43:00,002] Trial 0 finished with value: 1.2608137257746628 and parameters: {'learning_rate': 0.06554064060987876, 'max_depth': 14, 'num_boosted_rounds': 49, 'lambda': 0.7875049978766315, 'alpha': 0.3921881459782137, 'subsample': 0.6362963026413209, 'colsample_bytree': 0.6382321275715483}. Best is trial 0 with value: 1.2608137257746628.
[I 2024-04-05 18:43:00,430] Trial 1 finished with value: 0.9642321374914369 and parameters: {'learning_rate': 0.24254293148515557, 'max_depth': 20, 'num_boosted_rounds': 89, 'lambda': 0.364239097258288, 'alpha': 0.25548761150649474, 'subsample': 0.8417314675860681, 'colsample_bytree': 0.85635101349145}. Best is trial 1 with value: 0.9642321374914369.
[I 2024-04-05 18:43:00,796] Trial 2 finished with value: 1.0264000341682082 and parameters: {'learning_rate': 0.11737271888921452, 'max_depth': 13, 'num_boosted_rounds': 55, 'lam

Number of finished trials: 1000


In [15]:
import joblib
joblib.dump(study, 'models/xgboost_trial.pkl')

['models/xgboost_trial.pkl']

In [21]:
# get results
results = study.trials_dataframe(attrs=('number','value','duration', 'params'))
results = results.rename(columns={'value':'mae'})
results['duration'] = results['duration']/np.timedelta64(1, 's')
results = results.sort_values(by='mae',ascending=True)
results.to_csv(f'mae_xgboost_h_{h}.csv',index=False)
results.head(10)

,number,mae,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_num_boosted_rounds,params_subsample
874,874,0.471055,0.141757,0.193797,0.811638,0.193251,0.269102,9,53,0.713331
785,785,0.473332,0.113961,0.249292,0.840924,0.544477,0.228318,13,97,0.750251
796,796,0.474013,0.101845,0.283566,0.842758,0.659251,0.246838,7,91,0.764307
823,823,0.474103,0.106120,0.215805,0.788363,0.293164,0.275430,9,47,0.714899
531,531,0.474307,0.083769,0.266723,0.742482,0.510359,0.231371,13,10,0.692318
938,938,0.474723,0.117450,0.208316,0.828744,0.187908,0.271499,8,55,0.726878
699,699,0.475213,0.100660,0.250043,0.875186,0.175252,0.274779,12,98,0.748898
952,952,0.476170,0.108756,0.234223,0.844717,0.288706,0.276999,9,61,0.743631
509,509,0.476577,0.086255,0.271099,0.755896,0.135441,0.276053,12,98,0.706170
943,943,0.477465,0.119930,0.218378,0.791196,0.247882,0.260062,10,55,0.738464


# Predict and save:

In [18]:
best_params = {'learning_rate': 0.2907681843683012,
 'max_depth': 7,
 #'num_boosted_rounds': 100,
 'lambda': 0.5916745313772411,
 'alpha': 0.011239143280416843,
 'subsample': 0.7313441579055963,
 'colsample_bytree': 0.8529057716885262}

In [61]:
def train_test_split(X, y, h, split_date): 
    """
    Split the train and test set for forecasting

    """
    print(f'Horizon: {h}')
    print('------------------------')

    X_train = X[X.index <= split_date].iloc[:-h, :]
    X_test = X[~X.index.isin(X_train.index)]

    #y_train = y[y.index <= split_date][h:]
    y_test = y[y.index > split_date]
    N, T = len(X_train), len(X_test)
    
    return N, T, y_test

N, T, y_test = train_test_split(X = cat_df, y = HICP_cat, h = 2, split_date= train_test_split_date)

Horizon: 2
Training predictor period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Training dependent variable period: 1997-03-31 00:00:00 to 2015-12-31 00:00:00
Test predictor period: 2015-11-30 00:00:00 to 2022-12-31 00:00:00
Test dependent variable period: 2016-01-31 00:00:00 to 2023-07-31 00:00:00
--------------------------------------------


In [55]:
def generate_forecast(X, y, N, T, h, hyperparam, model):
    y_pred_series = []
    for i in range(2, T+2-h): #T+1-h
        X_train = X.iloc[:N+i-h, :]
        y_train = y.iloc[h:N+i, :]

        X_test = X.iloc[N-h+i:N-h+i+1, :] #:N+i+3
        y_test = y.iloc[N+i:N+i+1, :] #:N+i+3+h

        print(f'Training period - features: {X_train.index[0]} to {X_train.index[-1]}')
        print(f'Training period - target : {y_train.index[0]} to {y_train.index[-1]}')
        print(f'Test period - features: {X_test.index}')
        print(f'Test period - target : {y_test.index}')

        # Standard scale:
        # For all things

        #### More specific to its own model
        y_test_pred = model(X_train, X_test, y_train, y_test, hyperparam)

        print(f'Forecast: {y_test_pred}')
        print('-------------------------------------------------------')
        y_pred_series.extend(y_test_pred.tolist())
        if X_test.index[-1] == X.index[-1]:
            break
    return y_pred_series
    
y_pred_series_h_2 = generate_forecast(X = cat_df, y = HICP_cat, N = N, T = T, h = h, hyperparam=best_params)

Horizon: 2
------------------------
Training period - features: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Training period - target : 1997-03-31 00:00:00 to 2015-12-31 00:00:00
Test period - features: DatetimeIndex(['2015-11-30'], dtype='datetime64[ns]', name='date', freq=None)
Test period - target : DatetimeIndex(['2016-01-31'], dtype='datetime64[ns]', name='date', freq=None)
Forecast: 
[0.8332401]
-------------------------------------------------------
Training period - features: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training period - target : 1997-03-31 00:00:00 to 2016-01-31 00:00:00
Test period - features: DatetimeIndex(['2015-12-31'], dtype='datetime64[ns]', name='date', freq=None)
Test period - target : DatetimeIndex(['2016-02-29'], dtype='datetime64[ns]', name='date', freq=None)
Forecast: 
[1.33021]
-------------------------------------------------------
Training period - features: 1997-01-31 00:00:00 to 2015-12-31 00:00:00
Training period - target : 1997-03-31 00:00:00 to

In [60]:
# Create and save:

forecast_df = pd.DataFrame(columns=[f'xgb_h_{i}' for i in [1, 2, 3]], index = y_test[y_test.index < '2023-01-31'].index)
forecast_df.iloc[:, 1] = y_pred_series_h_2

save_forecast(forecast_df, cat_file_path=f'data/forecast_results/{cat}_forecast.csv')